# 1. GPT(Generative Pre-Training)

- GPT 모델은 2018년 6월에 OpenAI가 논문을 통해 처음 제안

  - [GPT1](http://www.cs.ubc.ca/~amuham01/LING530/papers/radford2018improving.pdf)
  - [GPT2](http://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf)
  - [GPT3](http://arxiv.org/abs/2005.14165)
- GPT도 unlabeled data로 부터 pre-train을 진행한 후 특정 downstream tast에 fine-tuning을 하는 모델
- Transformer의 decorder만 사용하는 구조


### 1-1. GPT 모델의 특징
- 사전학습에는 대규모의 unlabeled data를 사용하는데 unlabeled data에서 단어 수준이상의 정보를 얻는 것은 매우 힘듬. 또한 어떤 방법이 유용한 텍스트 표현을 배우는데 효과적인지 불분명함
- 사전학습 이후에도 어떤 방법이 fine-tuning에 가장 효과적인지 불문명
- 논문에서는 unsupervised pre-training과 supervised fine-tuning의 조합을 사용한 접근법을 제한
- 모델은 이미 효과가 검증된 2017년 공개된 transformer를 사용
- fine-turning에서의 미세 조정만으로 다양한 자연어 처리 작업에 적용할 수 있는 범용적인 표현을 사전학습에서 학습하는 것

### 1-2. GPT 모델 구조
- GPT는 Transformer의 변형인 multi-layer Transformer decoder만 사용
- 입력 문맥 token의 multi-headed self-attention을 적용한 후, token에 대한 출력 분포를 얻기 위해 positino-wise feedforward layer를 적용

### 1-3. GPT 모델 학습
- unsupervised pre-training
  - 대규모 코퍼스에서 unsupervised learning 으로 , 연어 모델을 학습
  - transformer 디코더를 사용하여 계속 next token prediction 학습하는 것
  - multi-layer Transformer decoder를 사용하여 입력 문맥 token에 amulti-head-self-attention을 적용한 후 목표 token에 대한 출력 분포를 얻기 위해 position-wise feedworward 적용
- supervised fine-tuning
  - 특정 작업에 대한 데이터로 모델을 fine-tuning
  - 파인튜닝 단계에서는 사전학습된 모델을 각테스트에 맞게 input과 label로 구성된 supervised dataset에 대해 학습
  - 결과를 테스크에 맞는 loss들을 결합

> 사전 학습은 next token
prediction이라는 language modeling으로 진행되었기 때문에 각 downstream task와 input 모양이 다를 수 밖에 없음

### 1-4. GTP 모델의 특징
- 모델 구조의 변형이 거의 없음, 이전의 사전학습 모델들은 fine-tuning 할 때 모델 구조를 변형해야 하는 문제점이 있었음
- 모델 구조를 변형하지 않고 linear layer를 마지막에 추가하는 아주 간단한 추가작업만 수행

# 2. GPT3f를 활용한 생성 실습

In [1]:
!pip install transformers

In [2]:
import torch
from transformers import AutoTokenizer, GPTJForCausalLM

In [3]:
# Hugging Face의 Transformer 라이브러리를 통해 한국어 GPT 모델
# BOS : 문장 시작
# EOS : 문자아끝
# UNK : 모르는단어
# PAD : 시퀀스의 길이를 맞출 떄 사용
# MASK : 모델 학습에 사용
tokenizer = AutoTokenizer.from_pretrained('MrBananaHuman/kogpt_6b_fp16',
                                          bos_token='[BOS]',
                                          eos_token='[EOS]',
                                          unk_token='[UNK]',
                                          pad_toekn='[PAD]',
                                          mask_token='[MASK]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

In [4]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.0 MB/s eta 0:00:00


In [5]:
# GPTJForCausalLM : OpenAI에서 개발한 GPT-J(GPT3 아키텍처)
# MrBananaHuman/kogpt_6b_fp16: 미리 학습된 가중치
# pad_token_id=tokenizer.eos_token_id: [EOS] 토큰을 사용
# low_cpu_mem_usage : CPU, 메모리 사용량을 최적화 하기 위한 플래그
model = GPTJForCausalLM.from_pretrained('MrBananaHuman/kogpt_6b_fp16',
                                        pad_token_id=tokenizer.eos_token_id,
                                        torch_dtype='auto',
                                        low_cpu_mem_usage=True)
model.to('cuda', non_blocking=True)
model.eval()


config.json:   0%|          | 0.00/984 [00:00<?, ?B/s]

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

* [GPT 디코딩 전략](https://littlefoxdiary.tistory.com/46)
- 문장을 생성할 때 beam search보다 sample 기반이 좋은 전략일 수 있음
- top-k/top-p 샘플링 전략이 좋으나, 동어 반복 문제는 생길 수 있음
- 다양한 방법을 실험해보며 사용자의 task에 적합한 전략을 선택하는 것이 중요

In [ ]:
# Beam Search 기반
prompt = '야! 너는 AI지? 정말로 말을 알아듣니?'
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        max_length=100,
        num_beams=5, # 1보다 큰 값을 지정
        no_repeat_ngram_size=2, # 2-gram의 어구가 반복되지 않도록 설정함
        num_return_sequences=5, # 5개의 문장을 리턴
        early_stopping=True # EOS 토큰이 나오면 생성을 중단
    )
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

In [ ]:
# Sampling 기반
prompt = '야! 너는 AI지? 정말로 말을 알아듣니?'
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        max_length=100,
        do_sample=True, # 샘플링 전략을 사용
        top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플리에서 제외
        top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
        temperature=0.75, # 낮으면 기존 단어를 재활용 확률이 높고, 높으면 random 확률이 증가
        num_return_sequences=3 # 3개의 결과를 디코딩함
    )
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)

In [ ]:
prompt = '''
요즘 굉장히 핫한 인공지능인 GPT님과 인터뷰를 나눠보겠습니다!
Q : GPT님, 요즘 바쁘시죠?
A : '''
with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        max_length=256,
        do_sample=True, # 샘플링 전략을 사용
        top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플리에서 제외
        top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
        temperature=0.85, # 낮으면 기존 단어를 재활용 확률이 높고, 높으면 random 확률이 감소
        num_return_sequences=3 # 3개의 결과를 디코딩함
    )
    generated = tokenizer.batch_decode(gen_tokens)[0]
print(generated)